# Rebuild All Events
    This script builds/rebuilds all the events of Live Mouse Tracker
    You can also create your own events and add them to this process
    
    Approximate time is 1h of computation for a 24h database of 4 animals


# Parameters:
    This will compute automatically events for your database, and adjust settings for you.
    
    If you remove the automatic settings, you can force parameters:
        Set minT and maxT to process the database.
        Set windowT to divide the computation load in segment of that duration. Default value is 1 day.

In [6]:
import sys
sys.path.insert(1, "../")
from lmtanalysis.Measure import *

# set this to false if you want to set manual parameters.
automaticSettings = True

# Manual parameters:

''' minT and maxT to process the analysis (expressed in frame) '''
minT = 0
maxT = 3*oneDay
''' time window to compute the events (in frame). '''
windowT = 1*oneDay
''' speed up process '''
USE_CACHE_LOAD_DETECTION_CACHE = True

# Run this section to compute your database
    You will be prompt to provide a database

In [7]:
''' Created on 26 march 2019 @author: Fab '''

import sys
sys.path.insert(1, "../")
import sqlite3
from lmtanalysis.Animal import *
import matplotlib.pyplot as plt
from lmtanalysis.Event import *
from lmtanalysis.Measure import *
from lmtanalysis import BuildEventTrain3, BuildEventTrain4, BuildEventTrain2, BuildEventFollowZone, BuildEventRear5, BuildEventFloorSniffing,\
    BuildEventSocialApproach, BuildEventSocialEscape, BuildEventApproachContact,BuildEventOralOralContact, BuildEventHuddling, \
    BuildEventApproachRear, BuildEventGroup2, BuildEventGroup3, BuildEventGroup4, BuildEventOralGenitalContact, \
    BuildEventStop, BuildEventNovObj_Interaction, \
    BuildEventMove, BuildEventGroup3MakeBreak, BuildEventGroup4MakeBreak,\
    BuildEventSideBySide, BuildEventSideBySideOpposite, BuildEventDetection,\
    BuildDataBaseIndex, BuildEventWallJump, BuildEventSAP,\
    BuildEventOralSideSequence, CheckWrongAnimal,\
    CorrectDetectionIntegrity, BuildEventNest4, BuildEventNest3, BuildEventGetAway
    
from psutil import virtual_memory

from tkinter.filedialog import askopenfilename
from lmtanalysis.TaskLogger import TaskLogger
import sys
import traceback
from lmtanalysis.FileUtil import getFilesToProcess
from lmtanalysis.EventTimeLineCache import flushEventTimeLineCache,\
    disableEventTimeLineCache


from lmtanalysis.EventTimeLineCache import EventTimeLineCached

def getNumberOfFrames(file):
    
    connection = sqlite3.connect( file )
    c = connection.cursor() 
    query = "SELECT MAX(FRAMENUMBER) FROM FRAME";
    c.execute( query )
    result = c.fetchall()
    numberOfFrames = int( result[0][0] )
    connection.close()
    
    return numberOfFrames

class FileProcessException(Exception):
    pass

eventClassList = [                  
                  BuildEventOralOralContact,
                  BuildEventOralGenitalContact,
                  BuildEventSideBySide,
                  BuildEventSideBySideOpposite,
                  BuildEventTrain2,                  
                  BuildEventTrain3,
                  BuildEventTrain4,
                  BuildEventMove,
                  BuildEventFollowZone,
                  BuildEventRear5,
                  BuildEventSocialApproach,
                  BuildEventGetAway,
                  BuildEventSocialEscape,
                  BuildEventApproachRear,
                  BuildEventGroup2,
                  BuildEventGroup3,
                  BuildEventGroup4,
                  BuildEventGroup3MakeBreak,
                  BuildEventGroup4MakeBreak,
                  BuildEventHuddling,
                  BuildEventStop,
                  BuildEventNovObj_Interaction,
                  BuildEventApproachContact,
                  BuildEventWallJump,
                  BuildEventSAP,
                  BuildEventOralSideSequence,
                  BuildEventNest3,
                  BuildEventNest4
                   ]

def flushEvents( connection ):
    
    print("Flushing events...")

    for ev in eventClassList:
        
        chrono = Chronometer( "Flushing event " + str(ev) )
        ev.flush( connection );      
        chrono.printTimeInS()
    

def processTimeWindow( connection, currentMinT , currentMaxT ):
    
    CheckWrongAnimal.check( connection, tmin=currentMinT, tmax=currentMaxT )
    
    # Warning: enabling this process (CorrectDetectionIntegrity) will alter the database permanently
    #CorrectDetectionIntegrity.correct( connection, tmin=0, tmax=maxT )
                            
    BuildEventDetection.reBuildEvent( connection, file, tmin=currentMinT, tmax=currentMaxT )

    animalPool = None
    
    flushEventTimeLineCache()
    
    if ( USE_CACHE_LOAD_DETECTION_CACHE ):
        print("Caching load of animal detection...")
        animalPool = AnimalPool( )
        animalPool.loadAnimals( connection )
        animalPool.loadDetection( start = currentMinT, end = currentMaxT )
        print("Caching load of animal detection done.")

    for ev in eventClassList:
        
        chrono = Chronometer( str( ev ) )
        ev.reBuildEvent( connection, file, tmin=currentMinT, tmax=currentMaxT, pool = animalPool )        
        chrono.printTimeInS()

def process( file ):

    print(file)
    
    if automaticSettings:
        print("Automatic settings.")
        windowT = 1*oneDay
        minT = 0
        maxT = getNumberOfFrames( file )
        print ( "Auto max set to" , maxT , "frames")
        
    
    chronoFullFile = Chronometer("File " + file )
    
    connection = sqlite3.connect( file )
    
    # update missing fields
    try:
        connection = sqlite3.connect( file )    
        c = connection.cursor()            
        query = "ALTER TABLE EVENT ADD METADATA TEXT";
        c.execute( query )    
        connection.commit()
        c.close()
        connection.close()
    except:
        print( "METADATA field already exists" , file )
        
    #build index
    
    BuildDataBaseIndex.buildDataBaseIndex( connection, force=False )
        
    '''
    # build sensor data    
    animalPool = AnimalPool( )
    animalPool.loadAnimals( connection )
    animalPool.buildSensorData(file)
    '''
    
    currentT = minT

    try:

        flushEvents( connection )
        
        while currentT < maxT:
                        
            currentMinT = currentT
            currentMaxT = currentT+ windowT
            if ( currentMaxT > maxT ):
                currentMaxT = maxT
                
            chronoTimeWindowFile = Chronometer("File "+ file+ " currentMinT: "+ str(currentMinT)+ " currentMaxT: " + str(currentMaxT) );
            processTimeWindow( connection, currentMinT, currentMaxT )    
            chronoTimeWindowFile.printTimeInS()
            
            currentT += windowT

                        

        print("Full file process time: ")
        chronoFullFile.printTimeInS()
        
        TEST_WINDOWING_COMPUTATION = False
        
        if ( TEST_WINDOWING_COMPUTATION ):
                
            print("*************")
            print("************* TEST START SECTION")
            print("************* Test if results are the same with or without the windowing.")
            
            # display and record to a file all events found, checking with rolling idA from None to 4. Save nbEvent and total len
            
            eventTimeLineList = []
            
            eventList = getAllEvents( connection )
            file = open("outEvent"+str(windowT)+".txt","w")  
            file.write( "Event name\nnb event\ntotal duration" )
            
            for eventName in eventList:
                for idAnimalA in range( 0,5 ):                
                        idA = idAnimalA 
                        if idA == 0:
                            idA = None
                        timeLine = EventTimeLineCached( connection, file, eventName, idA,  minFrame=minT, maxFrame=maxT )
                        eventTimeLineList.append( timeLine )
                        file.write( timeLine.eventNameWithId+"\t"+str(len(timeLine.eventList))+"\t"+str(timeLine.getTotalLength())+"\n" )            
            
            file.close() 
    
            #plotMultipleTimeLine( eventTimeLineList )
            
            print("************* END TEST")
        
        
    except:
        
        exc_type, exc_value, exc_traceback = sys.exc_info()
        lines = traceback.format_exception(exc_type, exc_value, exc_traceback)
        error = ''.join('!! ' + line for line in lines)
        
        t = TaskLogger( connection )
        t.addLog( error )
        
        print( error, file=sys.stderr ) 
        
        raise FileProcessException()
            
def getAllEvents( connection ):
    
    query = "select name from event group by name order by name"
    c = connection.cursor()     
    c.execute( query )    
    all_rows = c.fetchall()        
    header = [ "Name" ]
    data =[]    
    for row in all_rows:
        data.append( row[0] )
    return data        

print("Code launched.")

mem = virtual_memory()
availableMemoryGB = mem.total / 1000000000
print( "Total memory on computer: (GB)", availableMemoryGB ) 

if availableMemoryGB < 10:
    print( "Not enough memory to use cache load of events.")
    disableEventTimeLineCache()

print("A window is popping (maybe hidden) asking for files to process...")
files = getFilesToProcess()

chronoFullBatch = Chronometer("Full batch" )    

if ( files != None ):

    for file in files:
        try:
            print ( "Processing file" , file )
            process( file )
        except FileProcessException:
            print ( "STOP PROCESSING FILE " + file , file=sys.stderr  )

        flushEventTimeLineCache()

chronoFullBatch.printTimeInS()
print( "*** ALL JOBS DONE ***")

Code launched.
Total memory on computer: (GB) 16.984559616
A window is popping (maybe hidden) asking for files to process...
Processing file C:/Users/amyrh/Documents/AA_Neuroscience_MSc/AA_Project/Live Mouse Tracker/Processing Data/Project11_CorrectDetectionIdentityGraphs/Novel Object/19666-01-03-05_06-02-2020_Obj_Rebuilt_Uncorrected.sqlite
C:/Users/amyrh/Documents/AA_Neuroscience_MSc/AA_Project/Live Mouse Tracker/Processing Data/Project11_CorrectDetectionIdentityGraphs/Novel Object/19666-01-03-05_06-02-2020_Obj_Rebuilt_Uncorrected.sqlite
Automatic settings.
Auto max set to 332427 frames
METADATA field already exists C:/Users/amyrh/Documents/AA_Neuroscience_MSc/AA_Project/Live Mouse Tracker/Processing Data/Project11_CorrectDetectionIdentityGraphs/Novel Object/19666-01-03-05_06-02-2020_Obj_Rebuilt_Uncorrected.sqlite
Creating lmtanalysis indexes...
CREATE INDEX `animalIndex` ON `ANIMAL` (`ID` );
index animalIndex already exists
CREATE INDEX `detectionIndex` ON `DETECTION` (`ID` ASC,`FRAM

Number of event deleted: 0 
[Chrono  Flushing event <module 'lmtanalysis.BuildEventHuddling' from '..\\lmtanalysis\\BuildEventHuddling.py'>  ]  0.09474563598632812  seconds
Deleting event Stop in contact (idA:None,idB:None,idC:None,idD:None) from base...
DELETE FROM EVENT WHERE NAME="Stop in contact"
Number of event deleted: 0 
Deleting event Stop isolated (idA:None,idB:None,idC:None,idD:None) from base...
DELETE FROM EVENT WHERE NAME="Stop isolated"
Number of event deleted: 0 
[Chrono  Flushing event <module 'lmtanalysis.BuildEventStop' from '..\\lmtanalysis\\BuildEventStop.py'>  ]  0.17801189422607422  seconds
Deleting event NovObj Zone (idA:None,idB:None,idC:None,idD:None) from base...
DELETE FROM EVENT WHERE NAME="NovObj Zone"
Number of event deleted: 0 
Deleting event NovObj Stop (idA:None,idB:None,idC:None,idD:None) from base...
DELETE FROM EVENT WHERE NAME="NovObj Stop"
Number of event deleted: 0 
Deleting event NovObj Interact (idA:None,idB:None,idC:None,idD:None) from base...


Oral-oral Contact
Event Oral-oral Contact created. eventNameWithId = Oral-oral Contact idA:2 idB:3 idC:None idD:None loadEvent: False
Number of event:  1996
Mean length of event:  5.278557114228457
first event frame:  58
Keep previous entry.
Saving timeLine: Oral-oral Contact ( 1996 events )
Oral-oral Contact
Event Oral-oral Contact created. eventNameWithId = Oral-oral Contact idA:3 idB:1 idC:None idD:None loadEvent: False
Number of event:  2399
Mean length of event:  4.3772405168820345
first event frame:  122
Keep previous entry.
Saving timeLine: Oral-oral Contact ( 2399 events )
Oral-oral Contact
Event Oral-oral Contact created. eventNameWithId = Oral-oral Contact idA:3 idB:2 idC:None idD:None loadEvent: False
Number of event:  1996
Mean length of event:  5.278557114228457
first event frame:  58
Keep previous entry.
Saving timeLine: Oral-oral Contact ( 1996 events )
INSERT INTO LOG( process,version,date,tmin,tmax) VALUES ( 'Build Event Oral Oral Contact','0','2020-03-26 16:13:46','0'

Oral-genital Contact  Id( 2 , 1 , None , None ) Min/maxFrame: ( 0 / 332427 ) Loaded ( 2211  records loaded in  0.6146924495697021 S )
Caching eventTimeLine
Oral-genital Contact  Id( 2 , 3 , None , None ) Min/maxFrame: ( 0 / 332427 ) Loaded ( 2540  records loaded in  0.5117928981781006 S )
Caching eventTimeLine
Oral-genital Contact  Id( 3 , 1 , None , None ) Min/maxFrame: ( 0 / 332427 ) Loaded ( 2688  records loaded in  0.3965306282043457 S )
Caching eventTimeLine
Oral-genital Contact  Id( 3 , 2 , None , None ) Min/maxFrame: ( 0 / 332427 ) Loaded ( 1834  records loaded in  0.3411252498626709 S )
Caching eventTimeLine
Train2
Event Train2 created. eventNameWithId = Train2 idA:1 idB:2 idC:None idD:None loadEvent: False
Number of event:  12
Mean length of event:  7.916666666666667
first event frame:  6147
Keep previous entry.
Saving timeLine: Train2 ( 12 events )
Train2
Event Train2 created. eventNameWithId = Train2 idA:1 idB:3 idC:None idD:None loadEvent: False
Number of event:  9
Mean len

Event Move created. eventNameWithId = Move idA:3 idB:None idC:None idD:None loadEvent: False
Number of event:  5547
Mean length of event:  9.310257797007392
first event frame:  2
Keep previous entry.
Saving timeLine: Move ( 5547 events )
Event Move isolated created. eventNameWithId = Move isolated idA:3 idB:None idC:None idD:None loadEvent: False
Number of event:  3832
Mean length of event:  10.644050104384133
first event frame:  2
Keep previous entry.
Saving timeLine: Move isolated ( 3832 events )
Event Move in contact created. eventNameWithId = Move in contact idA:3 idB:None idC:None idD:None loadEvent: False
Number of event:  2552
Mean length of event:  4.253918495297806
first event frame:  53
Keep previous entry.
Saving timeLine: Move in contact ( 2552 events )
INSERT INTO LOG( process,version,date,tmin,tmax) VALUES ( 'Build Event Move','0','2020-03-26 16:14:23','0','332427' );
Rebuild event finished.
[Chrono  <module 'lmtanalysis.BuildEventMove' from '..\\lmtanalysis\\BuildEventMo

...

Group2  Id( 1 , None , None , None ) Min/maxFrame: ( 0 / 332427 ) Loaded ( 1221  records loaded in  1.1573376655578613 S )
Caching eventTimeLine
Contact  Id( 2 , None , None , None ) Loaded from cache ( 1107  records. )
Group2  Id( 2 , None , None , None ) Min/maxFrame: ( 0 / 332427 ) Loaded ( 1228  records loaded in  1.3554730415344238 S )
Caching eventTimeLine
Contact  Id( 3 , None , None , None ) Loaded from cache ( 1053  records. )
Group2  Id( 3 , None , None , None ) Min/maxFrame: ( 0 / 332427 ) Loaded ( 1106  records loaded in  1.4716835021972656 S )
Caching eventTimeLine
Contact  Id( 4 , None , None , None ) Min/maxFrame: ( 0 / 332427 ) Loaded ( 0  records loaded in  1.034109354019165 S )
Caching eventTimeLine
Group2  Id( 4 , None , None , None ) Min/maxFrame: ( 0 / 332427 ) Loaded ( 0  records loaded in  1.1044743061065674 S )
Caching eventTimeLine
Group4
Event Group4 created. eventNameWithId = Group4 idA:1 idB:2 idC:3 idD:4 loadEvent: False
Group4
Event Group4 created. eventNa

Number of event:  167
Mean length of event:  1.0
first event frame:  6194
Keep previous entry.
Saving timeLine: Group 3 make ( 167 events )
Number of event:  185
Mean length of event:  1.0
first event frame:  9105
Keep previous entry.
Saving timeLine: Group 3 break ( 185 events )
no event
Keep previous entry.
Saving timeLine: Group 3 make ( 0 events )
no event
Keep previous entry.
Saving timeLine: Group 3 break ( 0 events )
INSERT INTO LOG( process,version,date,tmin,tmax) VALUES ( 'Build Event Group3 Make Break','0','2020-03-26 16:16:48','0','332427' );
Rebuild event finished.
[Chrono  <module 'lmtanalysis.BuildEventGroup3MakeBreak' from '..\\lmtanalysis\\BuildEventGroup3MakeBreak.py'>  ]  28.92399287223816  seconds
Loading animals.
Fields available in lmtanalysis:  ['ID', 'RFID', 'GENOTYPE', 'NAME']
SQL Query: SELECT ID,RFID,NAME,GENOTYPE FROM ANIMAL ORDER BY GENOTYPE
Animal Id:1 Name:A RFID:000004849328 Genotype:None User1:None
Animal Id:2 Name:B RFID:000004849341 Genotype:None User1

current t 330000
Number of event:  3362
Mean length of event:  1.9003569303985723
first event frame:  826
Keep previous entry.
Saving timeLine: Huddling ( 3362 events )
Computing huddling for animal  Animal Id:3 Name:C RFID:000004849362 Genotype:None User1:None
Event Huddling created. eventNameWithId = Huddling idA:3 idB:None idC:None idD:None loadEvent: False
current t 0
current t 10000
current t 20000
current t 30000
current t 40000
current t 50000
current t 60000
current t 70000
current t 80000
Problem in Mask.getRoundness()
current t 90000
current t 100000
current t 110000
current t 120000
Problem in Mask.getRoundness()
Problem in Mask.getRoundness()
current t 130000
current t 140000
current t 150000
current t 160000
current t 170000
current t 180000
current t 190000
current t 200000
current t 210000
current t 220000
current t 230000
current t 240000
current t 250000
current t 260000
current t 270000
Problem in Mask.getRoundness()
current t 280000
Problem in Mask.getRoundness()
cur

INSERT INTO LOG( process,version,date,tmin,tmax) VALUES ( 'Build Event Novel Object','0','2020-03-26 17:20:19','0','332427' );
Rebuild event finished.
[Chrono  <module 'lmtanalysis.BuildEventNovObj_Interaction' from '..\\lmtanalysis\\BuildEventNovObj_Interaction.py'>  ]  9.716404914855957  seconds
Contact  Id( 1 , 2 , None , None ) Loaded from cache ( 1604  records. )
Social approach  Id( 1 , 2 , None , None ) Loaded from cache ( 15010  records. )
Contact  Id( 1 , 3 , None , None ) Loaded from cache ( 1905  records. )
Social approach  Id( 1 , 3 , None , None ) Loaded from cache ( 13329  records. )
Contact  Id( 2 , 1 , None , None ) Loaded from cache ( 1604  records. )
Social approach  Id( 2 , 1 , None , None ) Loaded from cache ( 18096  records. )
Contact  Id( 2 , 3 , None , None ) Loaded from cache ( 2715  records. )
Social approach  Id( 2 , 3 , None , None ) Loaded from cache ( 17397  records. )
Contact  Id( 3 , 1 , None , None ) Loaded from cache ( 1905  records. )
Social approach  

Number of event:  83
Mean length of event:  61.1566265060241
first event frame:  2891
Keep previous entry.
Saving timeLine: seq oral oral - oral genital ( 83 events )
Event seq oral oral - oral genital created. eventNameWithId = seq oral oral - oral genital idA:2 idB:3 idC:None idD:None loadEvent: False
seq oral oral - oral genital idA:2 idB:3 idC:None idD:None
Number of event:  53
Mean length of event:  53.16981132075472
first event frame:  8303
Keep previous entry.
Saving timeLine: seq oral oral - oral genital ( 53 events )
Event seq oral oral - oral genital created. eventNameWithId = seq oral oral - oral genital idA:3 idB:1 idC:None idD:None loadEvent: False
seq oral oral - oral genital idA:3 idB:1 idC:None idD:None
Number of event:  60
Mean length of event:  57.083333333333336
first event frame:  46760
Keep previous entry.
Saving timeLine: seq oral oral - oral genital ( 60 events )
Event seq oral oral - oral genital created. eventNameWithId = seq oral oral - oral genital idA:3 idB: